In [3]:
# Section 1: Setup and Data Loading
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
from pathlib import Path
import json

# Define custom model components
class CrossAttention(tf.keras.layers.Layer):
    """Custom layer implementing cross-attention between two feature vectors."""
    
    def __init__(self, num_heads=4, key_dim=16, **kwargs):
        super(CrossAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        
    def build(self, input_shape):
        self.query_dense = tf.keras.layers.Dense(self.num_heads * self.key_dim)
        self.key_dense = tf.keras.layers.Dense(self.num_heads * self.key_dim)
        self.value_dense = tf.keras.layers.Dense(self.num_heads * self.key_dim)
        self.output_dense = tf.keras.layers.Dense(input_shape[0][-1])
        super(CrossAttention, self).build(input_shape)
        
    def call(self, inputs):
        query, key_value = inputs
        query = self.query_dense(query)
        key = self.key_dense(key_value)
        value = self.value_dense(key_value)
        
        batch_size = tf.shape(query)[0]
        query = tf.reshape(query, [batch_size, 1, self.num_heads, self.key_dim])
        key = tf.reshape(key, [batch_size, 1, self.num_heads, self.key_dim])
        value = tf.reshape(value, [batch_size, 1, self.num_heads, self.key_dim])
        
        attention_scores = tf.matmul(query, key, transpose_b=True)
        attention_scores = attention_scores / tf.math.sqrt(tf.cast(self.key_dim, tf.float32))
        attention_weights = tf.nn.softmax(attention_scores, axis=-1)
        
        attention_output = tf.matmul(attention_weights, value)
        attention_output = tf.reshape(attention_output, [batch_size, self.num_heads * self.key_dim])
        output = self.output_dense(attention_output)
        
        return output
    
    def get_config(self):
        config = super(CrossAttention, self).get_config()
        config.update({
            'num_heads': self.num_heads,
            'key_dim': self.key_dim
        })
        return config

def custom_loss(y_true, y_pred):
    """Custom loss function combining MSE for each magnetic field component."""
    # Split predictions into components
    be_pred, bn_pred, bu_pred = tf.unstack(y_pred, axis=1)
    be_true, bn_true, bu_true = tf.unstack(y_true, axis=1)
    
    # Calculate MSE for each component
    be_loss = tf.keras.losses.mean_squared_error(be_true, be_pred)
    bn_loss = tf.keras.losses.mean_squared_error(bn_true, bn_pred)
    bu_loss = tf.keras.losses.mean_squared_error(bu_true, bu_pred)
    
    # Weight vertical component equally
    return be_loss + bn_loss + bu_loss

# Set up plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16

# Rest of the code remains the same...

# Set up plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16

# Define paths
DATA_DIR = "/Users/akv020/Tensorflow/fennomag-net/source/model2024/data"
MODEL_DIR = "/Users/akv020/Tensorflow/fennomag-net/source/model2024/model_outputs"
MAGNETOMETER_DIR = "/Users/akv020/Tensorflow/fennomag-net/data/XYZmagnetometer"

# Load model configuration
with open(os.path.join(MODEL_DIR, 'model_config.json'), 'r') as f:
    model_config = json.load(f)

# Load data statistics
with open(os.path.join(MODEL_DIR, 'data_statistics.json'), 'r') as f:
    data_stats = json.load(f)

# Print configuration
print("Model Configuration:")
print(f"Branch 1 Lookback: {model_config['temporal_params']['branch1_lookback']} timesteps")
print(f"Branch 2 Lookback: {model_config['temporal_params']['branch2_lookback']} timesteps")
print(f"Forecast Horizon: {model_config['temporal_params']['forecast_horizon']} minutes")
print(f"Batch Size: {model_config['temporal_params']['batch_size']}")

print("\nData Statistics:")
print(f"Total days: {data_stats['temporal_info']['total_days']}")
print(f"Samples per day: {data_stats['temporal_info']['samples_per_day']:.2f}")
print(f"Branch 1 interval: {data_stats['temporal_info']['branch1_interval']}")
print(f"Branch 2 interval: {data_stats['temporal_info']['branch2_interval']}")

# Load the trained model
print("\nLoading trained model...")
model = tf.keras.models.load_model(
    os.path.join(MODEL_DIR, 'best_model.h5'),
    custom_objects={'CrossAttention': CrossAttention, 'custom_loss': custom_loss}
)
model.summary()

# Load station coordinates
print("\nLoading station coordinates...")
station_coords = pd.read_csv(os.path.join(DATA_DIR, 'magnetometer/station_coordinates.csv'))
station_coords.set_index('station', inplace=True)
print(f"Loaded coordinates for {len(station_coords)} stations")

# Load grid metadata
print("\nLoading grid metadata...")
with open(os.path.join(DATA_DIR, '2024/grid_metadata.txt'), 'r') as f:
    grid_metadata = {}
    for line in f:
        key, value = line.strip().split(': ')
        try:
            if '[' in value and ']' in value:
                value = value.strip('[]').split()
                grid_metadata[key] = np.array([float(v) for v in value])
            else:
                grid_metadata[key] = float(value)
        except ValueError:
            grid_metadata[key] = value

# Find station nearest to grid center
grid_center = (grid_metadata['grid_center_lon'], grid_metadata['grid_center_lat'])
distances = np.sqrt(
    (station_coords['longitude'] - grid_center[0])**2 +
    (station_coords['latitude'] - grid_center[1])**2
)
nearest_station = station_coords.index[distances.argmin()]
print(f"\nNearest station to grid center: {nearest_station}")
print(f"Distance: {distances.min():.2f} degrees")
print(f"Coordinates: {station_coords.loc[nearest_station, 'longitude']:.2f}°E, "
      f"{station_coords.loc[nearest_station, 'latitude']:.2f}°N")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/akv020/Tensorflow/fennomag-net/source/model2024/model_config.json'